The Pure QR algorithm is a way of doing similarity transform on a matrix to a Hessenberg Triangular form. 


As we can see, we have the matrix approaching HessenBurg Triagular form after the iteration. But the interesting thing is, this QR algorithm is actually equivalent to simultaneous iteration! 

The simualtenous iterations is an augmentation of the power iteration method, it exploit the fact that, the magnitude of the eigenvalues can be different, which adds to its speed of convergence. The rate of convergen is 1st order tho. 

The key is to project the iterations vectors onto itself and keep them orthogonal. However it only really applies to Hermitian Matrix...




### This is a collections of all the methods we are going to use for this file. 

In [2]:
using LinearAlgebra
function RandHermitianMatrix(n)
    M = rand(n, n)
    M = triu(M)
    M = M + M'
    return M
end

function pureQR(itr::Int64, m::AbstractArray{<: Number})
    Ak = copy(m)
    Q = I
    for k = 1: itr
        F = qr(Ak)
        Ak = F.R*F.Q
        Q *= Q*F.Q
    end
    return Ak, Q
end

function SimultaneousItr(itr::Int64, m::AbstractArray{<:Number}) 
    Q = I
    A = copy(m)
    for II = 1: itr
        Z = A*Q
        F = qr(Z)  # Not Modified Gram schimt, will it still work?
        Q = F.Q        
    end
    return Q'*A*Q, Q
end

function RandTridiognalHermitian(n:: Int64)
    Result = zeros(n, n)
    for II = 1: n
        Result[II, II] = rand()
    end
    RandomArray = rand(1, n - 1)
    for JJ = 1: n - 1
        Result[JJ, JJ + 1] = RandomArray[JJ]
        Result[JJ + 1, JJ] = RandomArray[JJ]
    end
    return Result
end

function PureQRAuto(m::AbstractArray{<:Number}; Abstol:: Number=1e-6)
    if m' != m
        throw("Matrix is not hermitial, don't use PureQRAuto.")
    end 
    MaxItr = 1e4; Counter = 0;
    A = copy(m)
    Q = I
    while norm(A - Diagonal(A), Inf) > Abstol && Counter < MaxItr
        Decomp = qr(A)
        Q     *= Q*Decomp.Q
        A      = Decomp.R*Decomp.Q
        Counter += 1
    end
    print("Error infinity norm: ", norm(A - Diagonal(A), Inf))
    return diag(A), Q
end


PureQRAuto (generic function with 1 method)

In [34]:
M = rand(4, 4)
display(pureQR(10, M)[1])
display(pureQR(40, M)[1])


4×4 Array{Float64,2}:
  2.04018      -0.123925    -0.318286    -0.00543013
  7.93811e-5    0.725978     0.315815    -0.195839
 -1.80078e-10   2.93742e-5  -0.261866    -0.691081
 -3.06615e-12  -5.49906e-8   0.00119709   0.149206

4×4 Array{Float64,2}:
  2.04017      -0.124014     -0.318246     -0.00637333
  2.73976e-18   0.725995      0.316376     -0.194934
 -2.64104e-37   1.20946e-18  -0.259853     -0.692272
 -1.69915e-46  -8.82944e-29   4.65343e-11   0.147184

In [22]:
M    = RandHermitianMatrix(5)
D, V = SimultaneousItr(40, M)

display("SimultaneousItr Eigenvalues: ")
display(diag(D))
display("SimultaneousItr Eigenvectors: ")
display(V)
display("The correct answer from julia: ")
display(eigen(M))



"SimultaneousItr Eigenvalues: "

5-element Array{Float64,1}:
  2.5663651689132605
  1.2071520897836183
  0.6664262537374328
 -0.45854841833765525
 -0.06428848684924954

"SimultaneousItr Eigenvectors: "

5×5 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.424221  -0.178875   0.305945    0.154959     -0.818795
 -0.624608  -0.609728   0.0695223   0.000965087   0.482973
 -0.28528    0.138596  -0.792146    0.515125     -0.0809725
 -0.326829   0.159171  -0.38008    -0.834294     -0.165351
 -0.49163    0.742759   0.360011    0.120776      0.249827

"The correct answer from julia: "

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.45854841833765914
 -0.0642884868492502
  0.6664262537374357
  1.207152089783617
  2.5663651689132587
vectors:
5×5 Array{Float64,2}:
  0.154959      0.818795   -0.305945   -0.178875  -0.424221
  0.000965083  -0.482973   -0.0695223  -0.609728  -0.624608
  0.515126      0.0809725   0.792146    0.138596  -0.28528
 -0.834294      0.165351    0.38008     0.159171  -0.326829
  0.120776     -0.249827   -0.360011    0.742759  -0.49163

In [44]:
TridMatrix = RandTridiognalHermitian(5)
display(TridMatrix)
V, Q = SimultaneousItr(50, TridMatrix)
display(V)
display(eigen(TridMatrix))

5×5 Array{Float64,2}:
 0.874553  0.111751  0.0       0.0       0.0
 0.111751  0.90972   0.766485  0.0       0.0
 0.0       0.766485  0.362628  0.412029  0.0
 0.0       0.0       0.412029  0.833455  0.707387
 0.0       0.0       0.0       0.707387  0.303411

5×5 Array{Float64,2}:
  1.62208      -8.0205e-7    3.33067e-16   5.55112e-16   2.77556e-16
 -8.0205e-7     1.23045      6.77865e-8    0.0           1.11022e-16
 -6.66134e-16   6.77865e-8   0.86063       0.0           1.11022e-16
  2.02963e-16  -8.32667e-17  5.55112e-17  -0.405643      5.55112e-17
 -5.20417e-18  -1.73472e-18  1.82146e-16   1.31839e-16  -0.0237483

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.4056434362624408
 -0.023748345796204884
  0.8606296313406394
  1.2304491067142749
  1.6220800100758561
vectors:
5×5 Array{Float64,2}:
  0.0321679  -0.0529362   0.977941   -0.17929   0.0874581
 -0.368509    0.425524   -0.121846   -0.570989  0.585027
  0.627706   -0.510508   -0.134777   -0.212786  0.530964
 -0.4849     -0.312865    0.0637674   0.614022  0.534695
  0.48376     0.676479    0.0809525   0.468537  0.286832

In [47]:
TridMatrix = RandTridiognalHermitian(5)
display(TridMatrix)
V, Q = pureQR(50, TridMatrix)
display(V)
display(eigen(TridMatrix))

5×5 Array{Float64,2}:
 0.426829  0.281821   0.0       0.0       0.0
 0.281821  0.0978248  0.498723  0.0       0.0
 0.0       0.498723   0.770544  0.494067  0.0
 0.0       0.0        0.494067  0.813606  0.405266
 0.0       0.0        0.0       0.405266  0.595717

5×5 Array{Float64,2}:
  1.4754       -1.77873e-12   3.29946e-16  -1.1238e-16   -4.08299e-17
 -1.77849e-12   0.841116      1.41126e-11  -8.49139e-17   5.28133e-17
  0.0           1.41124e-11   0.506526     -5.31458e-11  -1.27755e-17
  0.0           0.0          -5.31454e-11  -0.311676      3.78059e-12
  0.0           0.0           0.0           3.78057e-12   0.193153

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -0.31167591187641275
  0.19315283824690033
  0.5065264109453169
  0.8411160011836564
  1.4754013195480111
vectors:
5×5 Array{Float64,2}:
 -0.298664  -0.355528   0.8332     -0.293058  0.065535
  0.782642   0.294792   0.235625   -0.430806  0.243837
 -0.473856   0.257251  -0.277735   -0.476467  0.636494
  0.247931  -0.598206  -0.0894304   0.366808  0.661917
 -0.110733   0.602221   0.406355    0.605768  0.304942

In [10]:
TridMatrix = RandTridiognalHermitian(100)
m = TridMatrix
D, Q = PureQRAuto(TridMatrix)
display(D)

100-element Array{Float64,1}:
  2.1542220720583996
  2.0411226916509504
  1.815244823372171
  1.756464501353936
  1.7444948094339803
  1.6950430809568315
  1.667593924551519
  1.6479687168751593
  1.6135627193779745
  1.5902112218348512
  1.5832249864744765
  1.5508234129569132
  1.4899651204135802
  ⋮
  0.18066581433023388
  0.17008663003718064
  0.15711483879191396
 -0.13031871639900122
 -0.11813522528562191
  0.10565700268284195
 -0.07574145735735949
 -0.07200593275778314
  0.032102424845557774
  0.02532465275746051
  0.016728086261432625
  0.0018086929058987848

Error infinity norm: 1.110988203134551e-6

However, the pure QR algorithm is hardly an improvement from the Power iterations. And here, we will present the Invers poewr equivalent to the pure QR algorithm. 

The Shifted QR algorithm. 